In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from projet3 import utils
from projet3 import ead



/Users/alisaidomar/openclassroom_projet3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,24,25,26,28,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


QUALITÉS NUTRITIONELLES D'UN REPAS À DESTINATION DES SPORTIFS
==========================================================
Pour un sportif être capable d'analyser la qualité de son repas et connaitre les
proportions des différent marconutriments et/ou micronutriments.

PRÉ-AMBULE
===========
Depuis la source de données accessible à l'addresse https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv, nous allons réaliser les tâches suivantes:
1. Analyser la qualité de la données ; boulons, complétude.
2. Analyse statisque univarié.
3. Analyse statisque bivarié.
4. Exploitation des données dans le but d'extraire les proportions en terme de macronutriments/micronutriments.


# ANALYSE QUALITATIVE DE LA DONNÉE


Version

In [2]:
utils.print_version()

VERSION librairies
Python version 3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)]
Numpy version 1.17.3
Pandas version 0.25.2
Matplotlib version 3.1.1
Seaborn version 0.9.0


In [3]:
# Données brutes
data_df = ead.get_data()

#Info
# les 10 premiere lignes

Extrait du dataset

In [ ]:
data_df.head(10)

Info complète du dataset

In [ ]:
data_df.info(verbose=True, null_counts=True)


Description

In [ ]:
data_df.describe()


Les lignes en doublons

In [ ]:
data_df[data_df.duplicated()==True]

Complétude des colonnes, nous allons cherche les colonnes dont la compètude est >= 70 % et celles <= 0.01%

In [ ]:
completude_df = ead.get_columns_completude()

In [ ]:
completude_gt_70 = completude_df[completude_df["Completude"] >= 70.0]

In [ ]:
completude_gt_70

In [ ]:
print(f'Dimension {completude_gt_70.shape}')


In [ ]:
completude_lt_001 = completude_df[completude_df["Completude"] < 0.01]

In [ ]:
completude_lt_001

In [ ]:
print(f"Dimension {completude_lt_001.shape}")


## Résultats
Données issues de l'open data disponible gratuitement. Elle est composé de plus d'un million d'entrée et 175 colonnes:
- pour 51 colonnes on dispose de moins des 0.01% des lignes de renseignées, ces colonnes peuvent être omises de notre analyse
- pour 26 colonnes on dispose de plus de 70% des lignes de renseignées
- 14 lignes sont en doublons
- 119 colonnes correspondes à des données quantitative dont seulement 50 pour lesquels on dispose de plus de 1000 lignes (0.01% du volumes) de renseignées

## Conclusion
Les premières analyses sont plutôt positives; on dispose de peu de doublons et on dispose de 26 colonnes pour lesquelles plus de 70% des lignes sont renseignées.


# ANALYSES STATISTIQUES UNIVARIÉES
Dans cette partie nous allons procéder à l'analyse univariée sur les colonnes retenues dans le cadre de notre analyse

## Choix des colonnes
Dans un premier temps, sont retenues les colonnes pour lesquelles 70% de lignes sont remplies

In [ ]:
completude_gt_70["Column"]

Seules les colonnes
```
68                energy_100g
70                   fat_100g
71         saturated-fat_100g
106        carbohydrates_100g
107               sugars_100g
117             proteins_100g
121                 salt_100g
122               sodium_100g
```
Correspondent à des données quantitatives. La colonnes 'energy_100g' ne correspond ni à un macronutiment ni à un micronutriment, celle ne sera pas prise en compte. Etant donné le cadre de notre étude,afin de connaitre la répartition des micronutriments, nous allons donc étendre notre analyse à certaines colonnes incluses dans:

In [ ]:
completude_df[(completude_df["Completude"] >= 10.0)&(completude_df["Completude"] < 70.0)]

On retient alors les colonnes suivantes
```
104	trans-fat_100g	14.303652
105	cholesterol_100g	14.382140
116	fiber_100g	29.860238
124	vitamin-a_100g	13.779704
129	vitamin-c_100g	14.132801
143	calcium_100g	14.473610
145	iron_100g	14.142711
```
On regrettera de ne pas pouvoir retenir les vitamines B, leur couverture étant trop faible.

In [ ]:
all_columns = [
    'calcium_100g','cholesterol_100g', 'trans-fat_100g', 'iron_100g',
    'vitamin-c_100g', 'vitamin-a_100g', 'fat_100g', 'saturated-fat_100g', 
    'sugars_100g', 'carbohydrates_100g', 'proteins_100g','salt_100g',  
    'sodium_100g'
]
categories_columns = ['code', 'product_name']

## Détection et traitement des données aberrantes.

In [ ]:
# Restrictions aux index étudiés
data_restricted_df = data_df[all_columns]
data_restricted_df.describe()

On constate des données < 0 et d'autre > 100g, on va donc procéder à des remplacements, les données qui semblent < 0 et > 100 seront remplacées par la valeur médiane de la serie alors que les données non numériques seront elle remplacées par la valeur 0. 

On procède à ce type de remplacement afin de conserver la même forme de distribution.

In [ ]:
for c in data_restricted_df.columns:
    data_restricted_df[c][data_restricted_df[c] <= 0] = data_restricted_df[c].median()
    data_restricted_df[data_restricted_df >= 100] = data_restricted_df[c].median()
    data_restricted_df[data_restricted_df.isna()] = 0
    data_restricted_df[data_restricted_df.isnull()] = 0
    

In [ ]:
# Vérification
data_restricted_df.describe()

Visualisation des boites à moustache

In [ ]:
n = len(data_restricted_df.columns)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))
for i, c in enumerate(data_restricted_df.columns):
    plt.sca(ax[i])
    try:
        ax_ = sns.boxplot(data=data_restricted_df[c])
        plt.title(c)
    except ValueError as error:
        print(error)

Résultats pour les colonnes, calcium_100g, cholesterol_100g,trans-fat_100g,iron_100g,vitamin-c_100g, vitamin-a_100g les données sont trop applaties, nous avons un problème d'échelle. 
Ces données seront traitées en mg pour 100g.

In [ ]:
data_restricted_df["calcium_100g"] *= 1000
data_restricted_df["cholesterol_100g"] *= 1000
data_restricted_df["trans-fat_100g"] *= 1000
data_restricted_df["vitamin-c_100g"] *= 1000
data_restricted_df["vitamin-a_100g"] *= 1000

Affichage des boites à moustache sans les outliers afin de pouvoir afficher les quartiles pour toutes les colonnes.

In [ ]:
n = len(data_restricted_df.columns)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))
for i, c in enumerate(data_restricted_df.columns):
    plt.sca(ax[i])
    try:
        y_values = data_restricted_df[c]
        # affichage sans les outilers
        ax_ = sns.boxplot(data=data_restricted_df[c], showfliers=False)
        plt.title(c)
        #plt.ylim(0, 110)
    except ValueError as error:
        print(error)

## Affichage des histogrammes pour les différents index

In [ ]:
# 
columns  = all_columns
data_restricted_df[columns][data_restricted_df[columns].isna()] = 0
n = len(columns)
fig, ax = plt.subplots(n, 1, figsize=(12,n*6))

for i, s in enumerate(columns):
    try:
        plt.sca(ax[i])
        print(f"Analyse de {s}")
        print(data_restricted_df[s].describe())
        sns.distplot(data_restricted_df[s], kde=True, norm_hist=True)
    except Exception as error:
        print(f'{error} while the display')


## Résultats
Une fois de plus on constate que les colonnes calcium_100g, cholesterol_100g,trans-fat_100g,iron_100g,vitamin-c_100g, vitamin-a_100g sont très applaties. Pour les autres index on constate une répartition asymétrique vers la droite avec une valeur modal assez proche de 0. 

## Conclusion
Etant donnée le cadre de notre projet; répartition des macronutriment et/ou micronutriments dans le cadre de d'une alimentation de sportif. 
Nous allons omettre ces colonnes afin de préciser notre projet et nous concentrer sur la répartition des macronutriments.


# ANALYSES STATISTIQUES MULTIVARIÉES

## Affichage des pair plots
Nous allons au préalable filter sur les lignes pour lesquelles toutes les colonnes sont renséignées. 
l' "energy_100g" pouvant etre calculée à partir des macronutriments.

In [ ]:
all_columns_mutivar = [
    'fat_100g', 'saturated-fat_100g', 
    'sugars_100g', 'carbohydrates_100g', 'proteins_100g','salt_100g',  
    'sodium_100g'
]
sns.pairplot(data_restricted_df[all_columns_mutivar]) 

## Affichage de la Heat map

In [ ]:
sns.heatmap(data_restricted_df[all_columns_mutivar].select_dtypes(include=np.number).corr())

## Résultats
On constate une forte corrélation etre le sodium et le sel, nous pouvons donc conserver uniquement l'une de ces colonnes.


# EXPLOITATION DES DONNEES
Analyse de la composition d'un repas, composé de plusiuers produits

Remarque:
En toute rigueur, il aurait fallu prendre en compte la 

In [4]:
columns = ['fat_100g', 'saturated-fat_100g', 'sugars_100g', 
           'carbohydrates_100g', 'proteins_100g','salt_100g']

Réalisation d'un échantillon (repas) compose de 5 produits

In [5]:
sample = data_df.sample(5)
codes = sample["code"]
sum_ = sample.sum()


Nom des produits 

In [6]:
sample[["product_name"]]

,product_name
584693,Olives noires à la Grecque
573105,Marrons Glacés Grand cru
734220,NaN
84495,100% Whole Wheat Bagels
707025,Tesco Raspberry Sponge Cake


Données de référence

In [ ]:
reference = utils.get_reference(data_df, columns, [])

In [ ]:
# Données de référence
reference

In [ ]:
# Données en Kcal
metrics_brut = utils.get_metrics(sample, codes=list(codes), columns=columns, columns_mg=[])
metrics_brut

Comparaison repas/reference

In [ ]:
sns.set_color_codes("pastel")
sns.set(style="whitegrid")
data_metrics = pd.DataFrame({"Repas": metrics_brut, "Reference": reference})
data_metrics.plot.barh()
plt.show()


Affichage radar

In [ ]:
utils.plot_radar(metrics_brut, columns)
utils.plot_radar(reference, columns)


Repartition energétique

In [ ]:

sample_c = utils.get_kcal(metrics_brut)
sample_c.describe()

Reparition energetique du repas

In [ ]:
sample_c.plot.pie(label="repas", autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2, figsize=(18, 9))

Reparition energetique de la référence

In [ ]:
sample_ref_c = utils.get_kcal(reference)
sample_ref_c.plot.pie(label="repas", autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2, figsize=(18, 9))

# CONCLUSION 
Dans la dernière partie nous nous sommes concentré à calculer la répartition des macronutiments pour 100g de produit, 